<a href="https://colab.research.google.com/github/DevAssis/LLM_RAG_Dados_LangChaim/blob/master/LLM_RAG_dados_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color=blue> LLM's com RAG

# Pipeline para Converter Dados Não Estruturados em Perguntas e Respostas

Este notebook implementa um pipeline de Processamento de Linguagem Natural (PNL) para transformar dados não estruturados em um sistema de perguntas e respostas interativo. O pipeline utiliza técnicas de embedding, armazenamento vetorial e modelos de linguagem avançados para permitir que os usuários obtenham insights a partir de seus dados.

## Etapas do Pipeline

1. **Carregamento de Dados**: Os dados não estruturados são carregados em um formato adequado para processamento.

2. **Divisão do Texto**: O texto é dividido em unidades menores para facilitar a análise e o gerenciamento.

3. **Embedding e Armazenamento**: As unidades de texto são convertidas em embeddings vetoriais e armazenadas em um banco de dados vetorial para busca eficiente.

4. **Recuperação e Geração**: O sistema recupera as informações relevantes do banco de dados e gera respostas concisas e informativas para as perguntas dos usuários.

# Etapa 1: Carregamento do Documento

Nesta etapa, as lib's são instaladas e os dados não estruturados são carregados utilizando o LangChain. A fonte de dados pode variar, incluindo arquivos de texto, páginas da web, etc. O carregador selecionado converte os dados brutos em um objeto `Document` do LangChain, que será utilizado nas etapas subsequentes.

**Exemplo:**


```
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://bkjfiel.com.br/proverbios-1")
data = loader.load()
```



In [ ]:
# Instalar o pacote atualizado
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successf

In [ ]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00


In [ ]:
# Instalar as bibliotecas necessárias
!pip install -U langchain langchain-community openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 4.3 MB/s eta 0:00:00


##### Fetching API Key from Environment Variable

In [ ]:
!pip install python-dotenv

In [1]:
import os
# Fetch the OpenAI API key from environment variables
api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
import os

# Verificar se a chave da API foi recuperada corretamente
api_key = os.getenv('OPENAI_API_KEY')

# Imprimir o resultado (por segurança, evite imprimir toda a chave publicamente)
print(api_key is not None)  # Deve retornar True se a chave foi carregada corretamente


False


In [ ]:
from langchain.document_loaders import WebBaseLoader

# Initialize the WebBaseLoader with the URL of the document to be loaded
loader = WebBaseLoader("https://bkjfiel.com.br/proverbios-1")

# Load the document and store it in the 'data' variable
data = loader.load()

In [ ]:
# Display the content of the loaded document
print(data)

[Document(metadata={'source': 'https://bkjfiel.com.br/proverbios-1', 'title': 'Provérbios 1 - Bíblia King James Fiel 1611', 'language': 'pt-BR'}, page_content='\n\n\n\n\nProvérbios 1 - Bíblia King James Fiel 1611\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n  Pular para o conteúdo\n\n\n\n\n\n\n\n\n\n\n\n\nMenu\n\n\n\n\n\nHome\nBíblia BKJ\n\nIntrodução a BKJ 1611\nConcordância Bíblica\nLivreto Explicativo\nApócrifos na KJV Inglês\nbvbooks.com.br\n\n\nFerramentas de Estudo\n\nVersículo do Dia\nVersículos Populares\nBíblia em um ano\n\n\nComunidade\n\nVoluntário BKJ\nTorne-se um cristão\nPedidos de Oração\n\n\nSobre\n\nSobre a BKJ\nSobre a BKJ em Inglês e Português\nSobre o Website\nFAQ: Perguntas Frequentes\nContato\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n      Busca avançada na Bíblia\n    \n\n\n\n\n\n\n\n\n\n\n\nEscolha um \xa0Livro\n          \n\nGênesis\nÊxodo\nLevítico\nNúmeros\nDeuteronômi

# Etapa 2: Dividindo o Documento em Partes

O documento carregado é dividido em partes menores, chamadas de "chunks". Essa divisão facilita o processamento e a geração de embeddings. O tamanho dos chunks e a sobreposição entre eles são definidos para garantir que o contexto seja mantido.

**Exemplo:**



```
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len
)

chunks = text_splitter.split_documents(data)
```



In [ ]:
import tiktoken

# Set up token encoding for the GPT-3.5 Turbo model
tiktoken.encoding_for_model('gpt-4o-mini')

<Encoding 'o200k_base'>

In [ ]:
tokenizer = tiktoken.get_encoding('o200k_base')

# Define a function to calculate the token length of a given text
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("O temor do Senhor é o princípio da Sabedoria.")

11

In [ ]:
# Este código inicializa um divisor de texto utilizando o `RecursiveCharacterTextSplitter`,
# que divide o texto em partes (chunks) de tamanho especificado.
# Neste caso, o tamanho do chunk é de 100 caracteres, com uma sobreposição de 20 caracteres entre os chunks.
# A função `length_function` é usada para calcular o comprimento do texto usando `tiktoken_len`.

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Inicializar o divisor de texto com parâmetros especificados
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len
)


In [ ]:
# Split the loaded document into smaller chunks
chunks = text_splitter.split_documents(data)

In [ ]:
# Check the total number of chunks generated
len(chunks)

23

# Etapa 3: Armazenando as Embeddings Vetoriais no Banco de Dados Vetorial

Cada chunk de texto é convertido em uma representação vetorial (embedding) usando um modelo de embedding pré-treinado. Essas embeddings capturam o significado semântico do texto e permitem a busca por similaridade. As embeddings e os chunks são armazenados em um banco de dados vetorial (ChromaDB) para recuperação eficiente.

**Exemplo:**



```
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

hf = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs = {'device' : 'cpu'}
)

vectordb = Chroma.from_documents(chunks, hf)
```


In [ ]:
# O código abaixo inicializa embeddings utilizando um modelo do HuggingFace.
# O modelo utilizado é o "sentence-transformers/all-MiniLM-L6-v2", que é um modelo leve e eficiente
# para gerar embeddings de sentenças.
# O parâmetro `model_kwargs` especifica que o modelo será executado na CPU,
# enquanto `encode_kwargs` define que as embeddings não serão normalizadas.
# Após a configuração dos parâmetros, o HuggingFace Embeddings é inicializado.

from langchain.embeddings import HuggingFaceEmbeddings

# Especificar o nome do modelo e argumentos adicionais
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device' : 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

# Inicializar as Embeddings do HuggingFace
hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)


<ipython-input-20-90183b84eae4>:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hf = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in y

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embed = hf.embed_documents(texts=['h','e'])

# Print the length of one of the embeddings to check its dimensions
print(len(embed[1]))

384


In [ ]:
# Install ChromaDB package using pip
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.5 MB/s eta 0:00:

In [ ]:
from langchain.vectorstores import Chroma

# Initialize Chroma vector database with chunks and HuggingFace embeddings
vectordb = Chroma.from_documents(chunks, hf)

In [ ]:
# Perform a similarity search on the vector database
vectordb.similarity_search('não mentir', k=3)

[Document(metadata={'language': 'pt-BR', 'source': 'https://bkjfiel.com.br/proverbios-1', 'title': 'Provérbios 1 - Bíblia King James Fiel 1611'}, page_content='13\n             encontraremos todos os bens preciosos, encheremos as nossas casas de despojos;\n          \n\n\n\n14\n             lança a tua sorte entre nós; tenhamos todos uma só bolsa.\n          \n\n\n\n15\n             Filho meu, não andes tu no caminho com eles; refreia o teu pé de suas veredas;\n          \n\n\n\n16\n             porque os seus pés correm para o mal, e se apressam a derramar sangue.'),
 Document(metadata={'language': 'pt-BR', 'source': 'https://bkjfiel.com.br/proverbios-1', 'title': 'Provérbios 1 - Bíblia King James Fiel 1611'}, page_content='Cantares de Salomão\nIsaías\nJeremias\nLamentações\nEzequiel\nDaniel\nOséias\nJoel\nAmós\nObadias\nJonas\nMiquéias\nNaum\nHabacuque\nSofonias\nAgeu\nZacarias\nMalaquias\nMateus\nMarcos\nLucas\nJoão\nAtos dos Apóstolos\nRomanos\n1 Coríntios'),
 Document(metadata={'l

# Etapa 4: Recuperar e Gerar

Quando o usuário faz uma pergunta, o sistema busca no banco de dados vetorial os chunks mais relevantes (semelhantes à pergunta). Um modelo de linguagem (LLM) utiliza esses chunks e a pergunta do usuário para gerar uma resposta completa e informativa.

**Exemplo:**


```
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name='gpt-4', temperature=0.6)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

response = qa_chain({'query': 'Como devo viver minha vida na terra?'})
```




In [ ]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Recuperar a chave da OpenAI da aba Secrets
api_key = userdata.get('OPENAI_API_KEY')

# Verificar se a chave foi carregada corretamente
if not api_key:
    raise ValueError("Chave da API OpenAI não encontrada. Verifique se foi configurada corretamente na aba Secrets.")

# Inicializar o modelo de linguagem com a chave diretamente
llm = ChatOpenAI(model_name='gpt-4', temperature=0.6, openai_api_key=api_key)

# Inicializar a cadeia RetrievalQA com o modelo de linguagem e o retriever do banco de dados vetorial
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

# Passar uma consulta para a cadeia de QA para gerar uma resposta
response = qa_chain({'query': 'Como devo viver minha vida na terra?'})

# Exibir a resposta
print(response)


<ipython-input-25-16c0b91b8660>:19: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = qa_chain({'query': 'Como devo viver minha vida na terra?'})


{'query': 'Como devo viver minha vida na terra?', 'result': 'De acordo com os versos apresentados, você deve viver sua vida buscando a sabedoria e o conhecimento, ouvindo instruções sábias e seguindo bons conselhos. É aconselhável evitar a ganância e caminhos que levem ao mal, e em vez disso, buscar a justiça e a retidão. Também é importante valorizar a instrução de seus pais e não abandonar seus ensinamentos.'}


In [ ]:
import pprint

# Exibir a resposta de forma mais organizada
pprint.pprint(response)


{'query': 'Como devo viver minha vida na terra?',
 'result': 'De acordo com os versos apresentados, você deve viver sua vida '
           'buscando a sabedoria e o conhecimento, ouvindo instruções sábias e '
           'seguindo bons conselhos. É aconselhável evitar a ganância e '
           'caminhos que levem ao mal, e em vez disso, buscar a justiça e a '
           'retidão. Também é importante valorizar a instrução de seus pais e '
           'não abandonar seus ensinamentos.'}


In [ ]:
#Change the query to what you want to ask the LLM
query = 'Como devo agir com as pessoas?'

In [ ]:
qa_chain({'query' : query})

{'query': 'Como devo agir com as pessoas?',
 'result': 'De acordo com os versos, você deve evitar pessoas que estão no caminho do mal e que se apressam para derramar sangue (verso 16). Em vez disso, você deve ouvir a instrução de seu pai e não abandonar a lei de sua mãe (verso 8), pois isso será como um ornamento de graça sobre sua cabeça e correntes ao seu pescoço (verso 9). Além disso, você deve se esforçar para aumentar seu aprendizado e buscar sábios conselhos (verso 5). O temor do Senhor é o princípio do conhecimento (verso 7), e você deve se esforçar para entender provérbios e suas interpretações, bem como as palavras dos sábios e seus enigmas (verso 6).'}

# Próximos Passos

* **Engenharia de Prompts**: Aperfeiçoar os prompts (instruções) fornecidos ao LLM para melhorar a qualidade e a relevância das respostas.

* **Prompt Templates**: Utilizar Prompt Templates do LangChain para criar prompts dinâmicos e reutilizáveis.

#FIM